In [ ]:
import pandas as pd

from src.prisma import prisma

In [ ]:
async def startup():
    await prisma.connect()

In [ ]:
async def shutdown():
    await prisma.disconnect()

In [ ]:
async def get_players_rounds_stats_by_match_id(match_id: str):
    match_rounds_raw_data = await prisma.round.find_many(
        where={
            'matchId': match_id,
        },
        include={
            'playerRoundStats': True
        },
    )

    match_rounds_players_stats = []
    for match_round_raw_data in match_rounds_raw_data:
        match_round_raw_data = match_round_raw_data.dict()

        match_round_players_stats = []
        for match_round_player_stats in match_round_raw_data["playerRoundStats"]:
            match_round_player_stats["matchId"] = match_round_raw_data["matchId"]
    
            match_round_players_stats.append(match_round_player_stats)

        match_rounds_players_stats += match_round_players_stats

    return match_rounds_players_stats

In [ ]:
async def get_players_rounds_stats_by_match_id_df(match_id: str):
    match_rounds_players_stats = await get_players_rounds_stats_by_match_id(match_id)

    match_rounds_players_stats_df = pd.DataFrame(match_rounds_players_stats)

    return match_rounds_players_stats_df

In [ ]:
async def get_players():
    players_raw_data = await prisma.player.find_many(
        distinct=["playerId"],
    )

    players = []
    for player_raw_data in players_raw_data:
        players.append(player_raw_data.dict())

    return players

In [ ]:
async def get_team_ids_player_ids_df():
    players = await get_players()

    players_df = pd.DataFrame(players)

    team_ids_player_ids_df = players_df[["playerId", "teamId"]]

    return team_ids_player_ids_df

In [ ]:
async def get_teams_rounds_stats_by_match_id(match_id: str):
    match_rounds_raw_data = await prisma.round.find_many(
        where={
            'matchId': match_id,
        },
        include={
            'teamRoundStats': True
        },
    )

    match_rounds_teams_stats = []
    for match_round_raw_data in match_rounds_raw_data:
        match_round_raw_data = match_round_raw_data.dict()

        match_round_teams_stats = []
        for match_round_team_stats in match_round_raw_data["teamRoundStats"]:
            match_round_team_stats["matchId"] = match_round_raw_data["matchId"]
    
            match_round_teams_stats.append(match_round_team_stats)

        match_rounds_teams_stats += match_round_teams_stats

    return match_rounds_teams_stats

In [ ]:
async def get_teams_rounds_stats_by_match_id_df(match_id: str):
    match_rounds_teams_stats = await get_teams_rounds_stats_by_match_id(match_id)

    match_rounds_teams_stats_df = pd.DataFrame(match_rounds_teams_stats)

    return match_rounds_teams_stats_df

In [ ]:
async def get_aces_by_match_id(match_id: str):
    match_rounds_players_stats_df = await get_players_rounds_stats_by_match_id_df(match_id)
    team_ids_player_ids_df = await get_team_ids_player_ids_df()

    match_rounds_players_stats_df = match_rounds_players_stats_df.merge(team_ids_player_ids_df, on="playerId", how="inner")

    team_ids = match_rounds_players_stats_df["teamId"].unique()

    match_rounds_players_stats_df["opponentTeamId"] = match_rounds_players_stats_df.apply(
        lambda x: team_ids[1] if x["teamId"] == team_ids[0] else team_ids[0],
        axis=1
    )

    num_players = match_rounds_players_stats_df.groupby(
        ["roundId", "teamId"]
    )["playerId"].count().reset_index().rename(columns={"playerId": "playersCount"})

    match_rounds_players_stats_df = match_rounds_players_stats_df.merge(
        num_players, 
        left_on=["roundId", "opponentTeamId"],
        right_on=["roundId", "teamId"],
        how="inner"
    )

    match_rounds_players_stats_df.rename(columns={"playersCount": "opponentTeamPlayersCount"}, inplace=True)

    match_rounds_players_stats_df["fractionOpponentTeamKilled"] = match_rounds_players_stats_df["kills"] / match_rounds_players_stats_df["opponentTeamPlayersCount"]

    match_rounds_players_stats_df["ace"] = (match_rounds_players_stats_df["fractionOpponentTeamKilled"] == 1.0)
    
    match_rounds_players_ace_df = match_rounds_players_stats_df[["roundId", "playerId", "fractionOpponentTeamKilled", "ace"]]

    return match_rounds_players_ace_df